In [1]:
import os
os.chdir(r"C:\Users\Aditya\GitHub\zavmo\zavmo-api\zavmo")
#os.chdir("/Users/adityachhabra/Github/zavmo/zavmo-api/zavmo")

In [2]:
from dotenv import load_dotenv
load_dotenv()
from IPython.display import Markdown

In [3]:
from helpers.agents.common import get_yaml_data,get_agent_instructions

In [4]:
from pydantic import BaseModel, Field, validator
from typing import Literal, List, Optional, Dict
from helpers.swarm import Agent, Response, Result, Tool, with_context, run_step
from helpers.agents.common import get_agent_instructions

In [ ]:
class update_discussion_data(Tool):
    interest_areas: str = Field(description="The learner's interest areas.")
    learning_style: str = Field(description="The learner's learning style.")
    curriculum: Curriculum = Field(description="The curriculum plan agreed upon by the learner.")
    timeline: str = Field(description="The learner's timeline.")

    def __str__(self):
        return "\n".join(f"{field.replace('_', ' ').title()}: {value}" 
                         for field, value in self.__dict__.items())
    @with_context
    def execute(self, context:Dict):
        

In [30]:
def transfer_to_delivery_agent():
    """Transfer to delivery agent after discussion stage data has been updated."""
    return None
    
#### For responding / handoff
discussion_agent = Agent(
    name="Discussion",
    model="gpt-4o-mini",
    instructions=get_agent_instructions('discuss'),
    functions=[
        request_curriculum,
        transfer_to_delivery_agent
    ],
    tool_choice="auto",
    parallel_tool_calls=False
)


In [5]:
### For handoff
def transfer_to_discussion_agent():
    """Transfer to the Discussion Agent when the learner is satisfied with the summary of the information gathered."""
    print("Transferring to Discussion Agent...")
    return discussion_agent

### For updating the data

class update_discover_data(Tool):
    """Update the learner's information gathered during the Discovery stage."""
    learning_goals: str = Field(description="The learner's learning goals.")
    learning_goal_rationale: str = Field(description="The learner's rationale for their learning goals.")
    knowledge_level: Literal["Beginner", "Intermediate", "Advanced", "Expert"] = Field(description="The learner's self-assessed knowledge level in their chosen area of study.")
    application_area: str = Field(description="A specific area or context where the learner plans to apply their new knowledge and skills.")
    
    def __str__(self):
        """Return a string representation of the UpdateDiscoverData object."""
        string = []
        for field, value in self.__dict__.items():
            string.append(f"{field}: {value}")
        return "\n".join(string)
    
    @with_context
    def execute(self, context: Dict):
        # Get email and sequence_id from context
        email       = context.get('email')
        sequence_id = context.get('sequence_id')
        if not email:
            raise ValueError("Email not found in context")
        if not sequence_id:
            raise ValueError("Sequence ID not found in context")
        
        # # Get the DiscoverStage object
        # discover_stage = DiscoverStage.objects.get(user_email=email, sequence_id=sequence_id)
        # if not discover_stage:
        #     raise ValueError("DiscoverStage not found")
        
        # # Update the DiscoverStage object
        # discover_stage.learning_goals = self.learning_goals
        # discover_stage.learning_goal_rationale = self.learning_goal_rationale
        # discover_stage.knowledge_level = self.knowledge_level
        # discover_stage.application_area = self.application_area
        # discover_stage.save()
        
        return Result(value=f"Updated DiscoverStage for {email} with sequence ID {sequence_id}. The following fields were updated: {str(self)}")
            

In [6]:
#### For responding / handoff
discover_agent = Agent(
    name="Discover",
    model="gpt-4o-mini",
    instructions=get_agent_instructions('discover'),
    functions=[
        update_discover_data,
        transfer_to_discussion_agent
    ],
    tool_choice="auto",
    parallel_tool_calls=False
)


In [7]:
agent    = discover_agent
messages = []
context  = {"email":"aditya@gmail.com","sequence_id":'52442'}

In [26]:
result = run_step(discover_agent, messages, context=context, max_turns=5)
messages.extend(result.messages)

2024-10-24 21:48:41,905 - INFO - Running step 0 with agent Discover
2024-10-24 21:48:43,290 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-24 21:48:43,292 - INFO - Updated context after update_discover_data call: {'email': 'aditya@gmail.com', 'sequence_id': '52442'}
2024-10-24 21:48:43,293 - INFO - Running step 1 with agent Discover


Input tokens: 1245
Output tokens: 46
Total tokens: 1291


2024-10-24 21:48:44,107 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-24 21:48:44,109 - INFO - Updated context after transfer_to_discussion_agent call: {'email': 'aditya@gmail.com', 'sequence_id': '52442'}
2024-10-24 21:48:44,110 - INFO - Running step 2 with agent Discover


Input tokens: 1355
Output tokens: 13
Total tokens: 1368
Transferring to Discussion Agent...


2024-10-24 21:48:45,500 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Input tokens: 1380
Output tokens: 66
Total tokens: 1446


In [27]:
Markdown(messages[-1]['content'])

Awesome! I've noted everything down. Now, I will hand you over to the next stage with a discussion agent, who will help you define specific learning objectives and create a personalized curriculum. Remember, your journey in data science is just beginning! 🚀

Good luck, Aditya, and enjoy your learning adventure! 🌟

In [25]:
messages.append({"role":"user","content":"great"})